<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_GPT_finetuning_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter
## (3 )Finetuning de GPT 4 mini para clasificación binaria

**Natalia Dedandi**



La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction


Guia para finetuning:

https://medium.com/@garethcull/fine-tuning-ai-models-a-practical-guide-for-beginners-dc313b2e0f76

In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install datasets seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      

In [3]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [4]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Levanto los datos de entrenamiento

In [5]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/36420 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11343 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/9106 [00:00<?, ? examples/s]

Tengo que transformar el DS de train en un conjunto de mensajes para pasarle a GPT para finetuning.

In [7]:
# prompt: Create a dataframe from ds1 with id text and hateful cols

df_train = pd.DataFrame(ds1['train'])[['id', 'text', 'HATEFUL']]


In [6]:
# prompt: create a datfram from ds1 for train

df_train = pd.DataFrame(ds1['train'])
df_train

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,343726,Video: salió de la cárcel por el coronavirus y...,@usuario Uno menos,Video: salió de la cárcel por el coronavirus y...,1,"Un hombre de 46 años, que cumplía una condena ...",0,0,0,0,0,0,0,0,1
1,384799,La muerte de Solange: su padre no pudo darle e...,@usuario #QueSeVayanTodos @usuario @usuario @u...,La carta que escribió Solange antes de morir: ...,0,"Solange Musse, la hija del hombre que había vi...",0,0,0,0,0,0,0,0,0
2,399435,Alberto Fernández negocia la compra de 15 mill...,"@usuario Te falta negociar con Cuba, mamerto.",Alberto Fernández negocia la compra de 15 mill...,0,Alberto Fernández desplegó una estrategia de a...,0,0,0,0,0,0,0,0,0
3,403171,Evo Morales dejó la Argentina con destino a Ve...,@usuario Se prepara para las milicias armadas,Evo Morales dejó la Argentina con destino a Ve...,0,"El ex presidente de Bolivia, Evo Morales, part...",0,0,0,0,0,0,0,0,0
4,394186,Les darán DNI provisorio a personas en situaci...,@usuario Seguimos alimentando vagos,Les darán DNI provisorio a personas en situaci...,1,"Una tarde lluviosa de marzo, cuando la pandemi...",0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36415,402634,Toma del Capitolio: Q-Shaman se niega a comer ...,@usuario Que se vaya a c...\n.,En prisión: Q-Shaman se niega a comer y su mam...,0,"Q-Shaman, uno de los seguidores del presidente...",0,0,0,0,0,0,0,0,0
36416,340909,Coronavirus en Alemania: la insólita factura d...,@usuario Me parece excelente,Coronavirus en Alemania: la insólita factura d...,1,Alemania parece estar entrando en una nueva fa...,0,0,0,1,0,0,0,0,0
36417,404621,Jujuy Jiménez filmó el momento en que era víct...,@usuario fuaahh todo es acoso . pobre de la ge...,Jujuy Jiménez registró el momento en qué era v...,0,Mientras disfruta de su incursión como panelis...,0,0,0,0,0,0,0,0,0
36418,342595,La furia de Coco Sily por la manifestación en ...,@usuario Si vas a jugar al póker todas las noc...,La furia de Coco Sily por la manifestación en ...,0,Coco Sily se encuentra internado con coronavir...,0,0,0,0,0,0,0,0,0


In [ ]:
df_hateful['sentiment_bin'] = df_hateful['sentiment_gpt'].apply(lambda x: 1 if (x=='hateful' or x=='Hateful') else 0)

In [ ]:


  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {
          'custom_id': str(row[id_column]),
          'method': 'POST',
          'url': '/v1/chat/completions',
          'body':{
              'model': 'gpt-4o-mini',
              'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row[text]},
                  {'role': 'assistant', 'content': row[text_column]}
                  ],
              'max_tokens': 20,  # Limit response to a single word
              'temperature': 0  # Keep response consistent
          }
      }
      for _, row in df.iterrows()
  ]


In [ ]:
def format_for_fine_tuning(data):


    training_data_list = []

    for x in data:

        updated_data = {
            "messages": [
                {
                    "role": "system",
                    "content": x['system_message']
                },
                {
                    "role": "user",
                    "content": x['user_content']
                },
                {
                    "role": "assistant",
                    "content": x['assistant_content']
                }
            ]
        }

        training_data_list.append(updated_data)

    print(training_data_list)

    return training_data_list

In [ ]:
training_tweets=format_for_fine_tuning(df_train_sample)
training_tweets

In [ ]:
#{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
def format_for_fine_tuning(data_tweet):

    training_data_list = []

    data_tweet = data_tweet.reset_index()  # make sure indexes pair with number of rows

    for index, row in data_tweet.iterrows():
        updated_data ={
            "prompt": row['prompt'],
            "completion": row['completion']}
        training_data_list.append(updated_data)

    return training_data_list